# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f50509d8a60>
├── 'a' --> tensor([[-0.8411, -1.3479,  0.7044],
│                   [ 1.2774,  0.6681, -0.8998]])
└── 'x' --> <FastTreeValue 0x7f5050a38e50>
    └── 'c' --> tensor([[ 0.3307, -0.3382,  0.3005,  0.2736],
                        [-0.2316, -0.5472, -0.9939, -0.9145],
                        [-0.5659,  0.1798,  0.6129,  0.2731]])

In [4]:
t.a

tensor([[-0.8411, -1.3479,  0.7044],
        [ 1.2774,  0.6681, -0.8998]])

In [5]:
%timeit t.a

74.9 ns ± 1.01 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f50509d8a60>
├── 'a' --> tensor([[ 2.2463,  1.4980, -0.2981],
│                   [-0.1827, -0.1013, -1.0766]])
└── 'x' --> <FastTreeValue 0x7f5050a38e50>
    └── 'c' --> tensor([[ 0.3307, -0.3382,  0.3005,  0.2736],
                        [-0.2316, -0.5472, -0.9939, -0.9145],
                        [-0.5659,  0.1798,  0.6129,  0.2731]])

In [7]:
%timeit t.a = new_value

73.2 ns ± 0.538 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8411, -1.3479,  0.7044],
               [ 1.2774,  0.6681, -0.8998]]),
    x: Batch(
           c: tensor([[ 0.3307, -0.3382,  0.3005,  0.2736],
                      [-0.2316, -0.5472, -0.9939, -0.9145],
                      [-0.5659,  0.1798,  0.6129,  0.2731]]),
       ),
)

In [10]:
b.a

tensor([[-0.8411, -1.3479,  0.7044],
        [ 1.2774,  0.6681, -0.8998]])

In [11]:
%timeit b.a

65.7 ns ± 1.31 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7304, -1.0835, -0.9274],
               [-1.8281,  0.2498,  1.3784]]),
    x: Batch(
           c: tensor([[ 0.3307, -0.3382,  0.3005,  0.2736],
                      [-0.2316, -0.5472, -0.9939, -0.9145],
                      [-0.5659,  0.1798,  0.6129,  0.2731]]),
       ),
)

In [13]:
%timeit b.a = new_value

613 ns ± 7.61 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1 µs ± 7.82 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.6 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

286 µs ± 5.61 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

273 µs ± 5.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5023f336a0>
├── 'a' --> tensor([[[-0.8411, -1.3479,  0.7044],
│                    [ 1.2774,  0.6681, -0.8998]],
│           
│                   [[-0.8411, -1.3479,  0.7044],
│                    [ 1.2774,  0.6681, -0.8998]],
│           
│                   [[-0.8411, -1.3479,  0.7044],
│                    [ 1.2774,  0.6681, -0.8998]],
│           
│                   [[-0.8411, -1.3479,  0.7044],
│                    [ 1.2774,  0.6681, -0.8998]],
│           
│                   [[-0.8411, -1.3479,  0.7044],
│                    [ 1.2774,  0.6681, -0.8998]],
│           
│                   [[-0.8411, -1.3479,  0.7044],
│                    [ 1.2774,  0.6681, -0.8998]],
│           
│                   [[-0.8411, -1.3479,  0.7044],
│                    [ 1.2774,  0.6681, -0.8998]],
│           
│                   [[-0.8411, -1.3479,  0.7044],
│                    [ 1.2774,  0.6681, -0.8998]]])
└── 'x' --> <FastTreeValue 0x7f50509d21f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

43.6 µs ± 684 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f50509df5b0>
├── 'a' --> tensor([[-0.8411, -1.3479,  0.7044],
│                   [ 1.2774,  0.6681, -0.8998],
│                   [-0.8411, -1.3479,  0.7044],
│                   [ 1.2774,  0.6681, -0.8998],
│                   [-0.8411, -1.3479,  0.7044],
│                   [ 1.2774,  0.6681, -0.8998],
│                   [-0.8411, -1.3479,  0.7044],
│                   [ 1.2774,  0.6681, -0.8998],
│                   [-0.8411, -1.3479,  0.7044],
│                   [ 1.2774,  0.6681, -0.8998],
│                   [-0.8411, -1.3479,  0.7044],
│                   [ 1.2774,  0.6681, -0.8998],
│                   [-0.8411, -1.3479,  0.7044],
│                   [ 1.2774,  0.6681, -0.8998],
│                   [-0.8411, -1.3479,  0.7044],
│                   [ 1.2774,  0.6681, -0.8998]])
└── 'x' --> <FastTreeValue 0x7f50509df6a0>
    └── 'c' --> tensor([[ 0.3307, -0.3382,  0.3005,  0.2736],
                        [-0.2316, -0.5472, -0.9939, -0.9145],
                 

In [23]:
%timeit t_cat(trees)

40.4 µs ± 589 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

80.8 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.3307, -0.3382,  0.3005,  0.2736],
                       [-0.2316, -0.5472, -0.9939, -0.9145],
                       [-0.5659,  0.1798,  0.6129,  0.2731]],
              
                      [[ 0.3307, -0.3382,  0.3005,  0.2736],
                       [-0.2316, -0.5472, -0.9939, -0.9145],
                       [-0.5659,  0.1798,  0.6129,  0.2731]],
              
                      [[ 0.3307, -0.3382,  0.3005,  0.2736],
                       [-0.2316, -0.5472, -0.9939, -0.9145],
                       [-0.5659,  0.1798,  0.6129,  0.2731]],
              
                      [[ 0.3307, -0.3382,  0.3005,  0.2736],
                       [-0.2316, -0.5472, -0.9939, -0.9145],
                       [-0.5659,  0.1798,  0.6129,  0.2731]],
              
                      [[ 0.3307, -0.3382,  0.3005,  0.2736],
                       [-0.2316, -0.5472, -0.9939, -0.9145],
                       [-0.5659,  0.1798,  0.6129,  0.2731]],

In [26]:
%timeit Batch.stack(batches)

101 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.3307, -0.3382,  0.3005,  0.2736],
                      [-0.2316, -0.5472, -0.9939, -0.9145],
                      [-0.5659,  0.1798,  0.6129,  0.2731],
                      [ 0.3307, -0.3382,  0.3005,  0.2736],
                      [-0.2316, -0.5472, -0.9939, -0.9145],
                      [-0.5659,  0.1798,  0.6129,  0.2731],
                      [ 0.3307, -0.3382,  0.3005,  0.2736],
                      [-0.2316, -0.5472, -0.9939, -0.9145],
                      [-0.5659,  0.1798,  0.6129,  0.2731],
                      [ 0.3307, -0.3382,  0.3005,  0.2736],
                      [-0.2316, -0.5472, -0.9939, -0.9145],
                      [-0.5659,  0.1798,  0.6129,  0.2731],
                      [ 0.3307, -0.3382,  0.3005,  0.2736],
                      [-0.2316, -0.5472, -0.9939, -0.9145],
                      [-0.5659,  0.1798,  0.6129,  0.2731],
                      [ 0.3307, -0.3382,  0.3005,  0.2736],
                   

In [28]:
%timeit Batch.cat(batches)

183 µs ± 2.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

632 µs ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
